# Task 5

This notebook is about getting same reusults as sql quires from database with pandas dataframes. In this I'll be using some of the basic pandas funcitons to find out same resutls as got in sql

##### Modules
Here I will import all the requried libraries to complete this particular task. Libraries that I'll be importing include pandas which is a library for dataframe manuplation, second module is sqlite3 which is a module I will be using to establish a sql database and add csv datasets into it, lastly I will import numpy which is a matrix library that can be used in this task.

In [313]:
import pandas as pd 
import numpy as np 
import sqlite3

In this part I am reading the 5 different datasets from new york flights data, these 5 datasets are about airlines, airports, flights, planes, and weather of new york flights. All these files are in csv, I will be readin them using pandas method `pd.read_csv()`

In [314]:
airlines= pd.read_csv("/kaggle/input/nyc-flights-data/airlines.csv")
airports = pd.read_csv("/kaggle/input/nyc-flights-data/airports.csv")
flights = pd.read_csv("/kaggle/input/nyc-flights-data/flights.csv")
planes = pd.read_csv("/kaggle/input/nyc-flights-data/planes.csv")
weather = pd.read_csv("/kaggle/input/nyc-flights-data/weather.csv")

After reading dataset it is mandatory to read the data and check if there is any nan values and remove them or handle with some other available ways, here I am putting 0 at these nan values if there are any available. 

In [315]:
airlines=airlines.fillna(0)
airports=airports.fillna(0)
flights=flights.fillna(0)
planes=planes.fillna(0)
weather=weather.fillna(0)

#### SQL database connection
Now that I have data, I need to establish a new database to store this data. I have here created a new databse that is `nyc_flights` and in this database I will now add all above pandas dataframes. Here I will use `sqlite3.connect()` method of library to establish a new connectin, and then I will also call `conn.cursor` which is a instance used to call database cursor function.

In [316]:
conn = sqlite3.connect('nyc_flights')
c = conn.cursor()

##### Adding tables 
Now that I have a new database I need to add tables from pandas dataframe, here I need to make tables in newly creaetd database I am executing sql table creation quires using `cursor.execute` which will exectue this query to create a new table with given table cloumn names. There will be new tables made each having these columns 

In [317]:
c.execute('CREATE TABLE IF NOT EXISTS Airlines (carrier text, name text)')
c.execute('CREATE TABLE IF NOT EXISTS Airports (faa text, name text, lat number, lon number, alt number, tx number, dst text,tzone text)')
c.execute('CREATE TABLE IF NOT EXISTS Flights (year number, month number,day number, sched_dep_time number,dep_delay number,arr_time number, sched_arr_time number, arr_delay number, carrier text, flight number, tailnum text, origin text, dest text, air_time number, distance number, hour number, minute number, time_hour time)')
c.execute('CREATE TABLE IF NOT EXISTS Weather (origin text,year number, month number,day number,hour number, temp number,humid number,wind_dir number, wind_speed number,wind_gust number precip number, pressure number, visib number, time_hour time)')
c.execute('CREATE TABLE IF NOT EXISTS Planes (tailnum text, year number, type text, manufacturer text, model text, engines number, seats number, speed number, engine text)')

#### Adding CSV files 
Now I have the tables in my databse and to add pandas frames I can use pandas `dataframe.to_sql()` method to convert the dataframe into sql table. In this all the rows of dataframe will be stored in above created tabels in specified tables columns. here `if_exists='replace'` I have used to replace the tables if there are already values in the table so that database does'nt rase the error. 

In [318]:
conn = sqlite3.connect('nyc_flights')
airports.to_sql('Airports', conn, if_exists='replace', index = False)
flights.to_sql('Flights', conn, if_exists='replace', index = False)
planes.to_sql('Planes', conn, if_exists='replace', index = False)
airlines.to_sql('Airlines', conn, if_exists='replace', index = False)
weather.to_sql('Weather', conn, if_exists='replace', index = False)


In [319]:
conn.commit()

#### SQL Quries with pandas dataframes

#### 1
In this first query I just need to find the unique values of a engine cloumn from planes table. I am using `pd.unique()` method from pandas which will give me exact results compared to sql query. 

In [320]:
res1a =pd.read_sql_query("""SELECT DISTINCT engine FROM Planes""",conn)
# Solution 
res1b = planes['engine'].unique()
res1b = pd.DataFrame(res1b,columns=['engine'])

In [321]:
res =pd.testing.assert_frame_equal(res1a, res1b)  

These are the distint engines availabele inside the planes

#### 2
In this particular query, I want to find the distint type and engine of planes from planes dataset. To find this I can use numpy `np.unique()` method which is used to extract unique values from dataframe columns and return an array of unique values.

In [322]:
res2a = pd.read_sql_query("""SELECT DISTINCT type, engine FROM Planes""",conn)
# Solution
res2b = planes.drop_duplicates(['type','engine'])[['type','engine']]
res2b = res2b.reset_index(drop=True)

In [323]:
pd.testing.assert_frame_equal(res2a, res2b)  

Above is the array of unique values from planes dataset with type and engine unique

#### 3
In this part I have to find out count and engine from planes dataset which should be grouped by engine column of planes dataset. In pandas this can be completed using `pd.groupby()` method which will group by the data and the at the end we can use a count() function to count the values of particular column

In [324]:
res3a= pd.read_sql_query("""SELECT COUNT(*), engine FROM Planes GROUP BY engine""",conn)
# Solution
res3b= planes.groupby('engine')['engine'].count().reset_index(name='COUNT(*)')
res3b =res3b.iloc[:, [1,0]]

In [325]:
pd.testing.assert_frame_equal(res3a, res3b)  

So here is the count of engine which is grouped by engine from planes dataset. 

#### 4
Here I am finding count, engine, and type from planes dataset and then grouping by the result with two columns names engine and type. This can be done using groupby function of pandas which will groupby the data and then at the end I will use a count() function to count the data. 

In [326]:
res4a = pd.read_sql_query('''SELECT COUNT(*), engine, type FROM Planes GROUP BY engine, type''',conn)
# Solution
res4b = planes.groupby(['engine', 'type'])['engine'].count().reset_index(name='COUNT(*)')
res4b =res4b.iloc[:, [2,0,1]]

In [327]:
pd.testing.assert_frame_equal(res4a, res4b)  

This above table is a grouped data based on enigne and type and their total sum.

#### 5
This query is about finding maximum, minimum, and average values of a column form dataframe with that I have to select engine and manufacturer column of the table and at the last data should be grouped by these two columns. To complete this using pandas I will be using pandas groupby() functino and the at end I'll use `pd.groupby().aggregate()` function to find these aggregate function values which are mean, max, and min for the year column of dataframe. 

In [328]:
res5a = pd.read_sql_query('''SELECT MIN(year), AVG(year), MAX(year), engine, manufacturer
FROM Planes
GROUP BY engine, manufacturer''',conn)
# Solution
res5b=planes.groupby(['engine','manufacturer']).agg(MIN=('year', 'min'),AVG=('year', 'mean'),MAX=('year','max')).reset_index()
res5b.rename(columns = {'MIN':'MIN(year)', 'MAX':'MAX(year)','AVG':'AVG(year)'}, inplace = True)
res5b = res5b.iloc[:, [2,3,4,0,1]]

In [329]:
pd.testing.assert_frame_equal(res5a, res5b)  

Above is the table of result that shows the engine and manufacturer having aggregate values of year column 

#### 6
In this query everything is to be extracted from planes dataset with a condition that there is no row with null values in speed column. This can be completed in pandas using `pd.notnull()` function , here I will use `pd.loc()` function to find only those rows which does not contain null values in the speed column. 

In [330]:
res6a = pd.read_sql_query('''SELECT * FROM planes WHERE speed IS NOT NULL''',conn)
# Solution
res6b=planes.loc[planes['speed'].notnull()]

In [331]:
pd.testing.assert_frame_equal(res6a, res6b)  

Thes are first 5 rows of dataset having no null values in speed column.

#### 7
Here tailnum column of planes dataset is to be selected and in this a condition is to set that number of seats should be greated then 150 and less then 190, and year should be greater then 2012. To find this I'll use `pd.loc()` function of pandas to find the rows and the give conditions usnig bitwise comparison function of python.

In [332]:
res7a =pd.read_sql_query('''SELECT tailnum FROM planes
WHERE seats BETWEEN 150 AND 190 AND year >= 2012''',conn)
# Solution
res7b =planes.loc[(planes['seats'] > 150) & (planes['seats'] < 190) & (planes['year']>=2012)]
col =res7b['tailnum']
res7b= col.to_frame().reset_index(drop=True)

In [333]:
pd.testing.assert_frame_equal(res7a, res7b)  

Thse are the tailnum values from planes dataframe having seats in between 150-190 and year having greater than 2012

#### 8
Here tailnum, manufacturer, seats column of planes dataset are selected having a condition that manufecturer should be from given list and seats should be greater then 390. Pandas has a function `pd.isin()` which takes a list as argument to extract only those rows which has these values form given list, and at the end I have used pd.loc() function to get those values having seats>390

In [334]:
res8a =pd.read_sql_query('''SELECT tailnum, manufacturer, seats FROM planes WHERE manufacturer IN ("BOEING", "AIRBUS", "EMBRAER") AND seats>390''',conn)
# Solution
res8b =planes[['tailnum','manufacturer','seats']][planes['manufacturer'].isin(["BOEING", "AIRBUS", "EMBRAER"])].loc[planes['seats']>390]
res8b=res8a.reset_index(drop=True)

In [335]:
pd.testing.assert_frame_equal(res8a, res8b)  

Above are the values which meet the requierments of query from planes dataset. 

#### 9
Now, here I have to select distint year and seats from planes datset, in which year should be greater then 2012 and results shoud be in order by ascending order according to the year and then according to the seats with discending order. I have used `drop_duplicates()` function of pandas dataframe to select only distint values of dataframe and giving it a argument `subset='year'` which will give a column with unique values and I'll do the same to find distinct valued of other column as a dataframe, after getting this I'll use pd.loc() method to set a condition that year>2012 and from this I'll have data having disting values greater than year 2012. At last to sort the data I am using `pd.sort_values` giving the arugument `by='column'` to sort by given column name and giving argument `ascending=True` to make it in asceding order, giving False to make it descending order.

In [336]:
res9a =pd.read_sql_query('''SELECT DISTINCT year, seats FROM planes
WHERE year >= 2012 ORDER BY year ASC, seats DESC''',conn)
# Soltuon
unique=planes.drop_duplicates(['year','seats'])
yeargreater = unique.loc[(unique['year']>=2012)]
res9b = yeargreater.sort_values(['year', 'seats'],ascending = [True, False])
res9b=res9b[['year','seats']]
res9b=res9b.reset_index(drop=True)

In [337]:
pd.testing.assert_frame_equal(res9a, res9b)  

#### 10
In this query I am chaning the order of sorting in the data by sorting seats first and then years. All other condition are same as above query

In [338]:
res10a =pd.read_sql_query('''SELECT DISTINCT year, seats FROM planes WHERE year >= 2012 ORDER BY seats DESC, year ASC''',conn)
# Solution
unique=planes.drop_duplicates(['year','seats'])
yeargreater = unique.loc[(unique['year']>=2012)]
res10b = yeargreater.sort_values(['seats', 'year'],ascending = [False, True])
res10b=res10b[['year','seats']]
res10b=res10b.reset_index(drop=True)

In [339]:
pd.testing.assert_frame_equal(res10a, res10b)  

#### 11
In this part I am finding manufacturer, and count of them which is grouped by manufacturer and having seats greater then 200. This can be completed by pandas groupby() method and at the end counting it. 

In [340]:
res11a =pd.read_sql_query('''SELECT manufacturer, COUNT(*) FROM planes WHERE seats > 200 GROUP BY manufacturer''',conn)
# Solution
res11b =planes[planes['seats'] > 200].groupby('manufacturer').count()
res11b = res11b.reset_index()
res11b = res11b[['manufacturer','tailnum']]


In [341]:
res11b=res11b.rename(columns={'tailnum':'COUNT(*)'})

In [342]:
pd.testing.assert_frame_equal(res11a, res11b)  

#### 12
This is the query about selecting manufacturer and count from planes and having a condition that count should be greater then 10. To complete this I will use groupby() funciton of pandas dataframe and then `groupby.filter()` method to filter only values of count aggregate function which are greated than 10, at the end I will use count function to count the values of groupby result.

In [343]:
res12a =pd.read_sql_query(''' SELECT manufacturer, COUNT(*) FROM planes GROUP BY manufacturer HAVING COUNT(*) > 10''',conn)
# Solution
res12b =planes.groupby('manufacturer').filter(lambda x: x['seats'].count() > 10).groupby('manufacturer')['seats'].count()

In [344]:
res12b = res12b.to_frame().reset_index()
res12b = res12b.rename(columns={'seats':'COUNT(*)'})

In [345]:
pd.testing.assert_frame_equal(res12a, res12b)  

#### 13
In this part I am selecting manufacturer, count from planes with a condition that seats are greater than 200 and count is greated than 10 finally grouping the data with manufacturer. To complete this using pandas dataframe, firt I am selecting only data that has seats greater than 200,then I am using gorupby() and filter() funciton of pandas dataframe to find the value of data having count greater then 10 and grouped by manufacturer, to find the count I am using count() funciton at the end. 

In [346]:
res13a =pd.read_sql_query('''SELECT manufacturer, COUNT(*) FROM planes WHERE seats > 200 GROUP BY manufacturer HAVING COUNT(*) > 10''',conn)
# Soluion
res13b =planes[planes['seats'] > 200].groupby('manufacturer').filter(lambda x: x['seats'].count() > 10).groupby('manufacturer')['seats'].count()

In [347]:
res13b= res13b.to_frame().reset_index()
res13b = res13b.rename(columns={'seats':'COUNT(*)'})

In [348]:
pd.testing.assert_frame_equal(res13a, res13b)  

#### 14
Here in this particular query I am selecting manefactuer, count and aliasing the count using howmany, and the data should be grouped by manufacturer, ordered by howmany and limit is 10. I am using aggregate function of pandas to find the count and then alias it with howmany, at last I will sort values using sort_values() method form pandas dataframe, and showing only first 5 elements. 

In [349]:
res14a =pd.read_sql_query('''SELECT manufacturer, COUNT(*) AS howmany
FROM planes
GROUP BY manufacturer
ORDER BY howmany DESC LIMIT 5
''',conn)
# Solution
res14b =planes.groupby('manufacturer').agg(howmany=('seats', 'count')).sort_values('howmany', ascending=False).head(5)

In [350]:
res14b = res14b.reset_index()

In [351]:
pd.testing.assert_frame_equal(res14a, res14b)  

#### 15
In this part I am using pandas `pd.merge()` method to merege two datasets with left join. I am puting `left_on='column'` and `right_on='column'` to merge the data based on this column, I am giving `how='left'` which joins the tables left join based, after completing this I am using `pd.rename()` to rename the column as alias the column names.

In [352]:
res15a =pd.read_sql_query('''SELECT
flights.*,
planes.year AS plane_year,
planes.speed AS plane_speed,
planes.seats AS plane_seats
FROM flights LEFT JOIN planes ON flights.tailnum=planes.tailnum''',conn)

# Solution
res15b= pd.merge(flights, planes[['year','speed','seats','tailnum']],on='tailnum', how="left")
res15b.rename(columns = {'year_x':'year','year_y':'plane_year','speed':'plane_speed','seats':'plane_seats'}, inplace = True)
res15a.drop('tailnum', inplace=True, axis=1)
res15b.drop('tailnum', inplace=True, axis=1)

In [353]:
pd.testing.assert_frame_equal(res15a,res15b)  

#### 16
In this query I am mergin two dataframes based on some third cumton dataframe, I am using `pd.drop_duplicates()` function to find the unique values of data, and then I am using this sorted data to store in a custom dataset names curtail which contain only two columns, after this I am mergin two differet dataframes inner join based with a column form curtail dataframe

In [354]:
res16a=pd.read_sql_query('''SELECT planes.*, airlines.* FROM
(SELECT DISTINCT carrier, tailnum FROM flights) AS cartail
INNER JOIN planes ON cartail.tailnum=planes.tailnum
INNER JOIN airlines ON cartail.carrier=airlines.carrier''',conn)

# Solution 
res16b = pd.merge(cartail, planes, left_on=cartail.tailnum, right_on=planes.tailnum).merge(airlines,on='carrier',how='inner')
res16b.drop('key_0',inplace=True,axis=1)
res16b.drop('tailnum_y',inplace=True,axis=1)
res16b = res16b.rename(columns={'tailnum_x':'tailnum'})
res16b = res16b.iloc[:, [1,2,3,4,5,6,7,8,9,0,10]]
res16a=res16a.sort_values(by=["carrier", "tailnum",'year','type','manufacturer','model','engines','seats','speed','engine','carrier','name'], ascending=False)[["carrier", "tailnum",'year','type','manufacturer','model','engines','seats','speed','engine','carrier','name']]
res16b=res16b.sort_values(by=["carrier", "tailnum",'year','type','manufacturer','model','engines','seats','speed','engine','carrier','name'], ascending=False)[["carrier", "tailnum",'year','type','manufacturer','model','engines','seats','speed','engine','carrier','name']]
res16a= res16a.reset_index(drop=True)
res16b=res16b.reset_index(drop=True)

In [355]:
pd.testing.assert_frame_equal(res16a, res16b)